In [1]:
# ========================================================================
# CELL 2: FMU SIMULATION WITH WEATHER DATA (UPDATED FOR NEW MODEL)
# ========================================================================
from fmpy import simulate_fmu
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# ============================================
# 1. LOAD WEATHER DATA
# ============================================
def load_weather_from_csv(csv_path):
    '''Load weather data from converted CSV (from EPW format)'''
    df = pd.read_csv(csv_path)
    
    print(f"✅ Loaded {len(df)} weather timesteps from {csv_path}")
    print(f"\nColumns in file: {df.columns.tolist()}")
    print(f"\nSample data:")
    print(df.head())
    
    return df


# ============================================
# 2. CREATE INPUT NUMPY STRUCTURED ARRAY FOR FMU
# ============================================
def prepare_fmu_input(weather_df):
    '''
    Convert weather DataFrame to numpy structured array for FMPy.
    
    ⚠️ FMPy REQUIRES input as numpy structured array, NOT DataFrame!
    
    INPUT VARIABLES FOR NEW MODEL (per HVAC_FMU_FIXED_INIT-5.mo):
    - TDryBul_in: Dry bulb temperature [K]
    - relHum_in: Relative humidity [0-1]
    - pAtm_in: Atmospheric pressure [Pa]
    - HGloHor_in: Global horizontal radiation [W/m²]
    - HDifHor_in: Diffuse horizontal radiation [W/m²]
    - winSpe_in: Wind speed [m/s]
    - uFan: Supply fan control signal [0-1]
    - uOA: Outside air damper signal [0-1]
    - uChiller: Chiller control signal [0-1]
    - uHeater: Heater control signal [0-1]
    - uFanEA: Exhaust fan signal [0-1]
    - occupancy: Occupancy ratio [0-1]
    '''
    
    # Calculate occupancy by hour (8:00-18:00)
    hours = (weather_df['time'] / 3600) % 24
    occupancy = np.where((hours >= 8) & (hours <= 18), 1.0, 0.0)
    
    # Create DataFrame with all inputs
    input_df = pd.DataFrame({
        'time': weather_df['time'],
        'TDryBul_in': weather_df['TDryBul'],
        'relHum_in': weather_df['relHum'],
        'pAtm_in': weather_df['pAtm'],
        'HGloHor_in': weather_df['HGloHor'],
        'HDifHor_in': weather_df['HDifHor'],
        'winSpe_in': weather_df['winSpe'],
        'uFan': 0.8,           # 80% fan speed
        'uOA': 0.9,            # 30% outside air
        'uChiller': 1.0,       # Chiller fully on
        'uHeater': 0.6,        # Heater off
        'uFanEA': 0.8,         # 30% exhaust fan
        'occupancy': occupancy
    })
    
    # ✅ CONVERT DATAFRAME TO NUMPY STRUCTURED ARRAY
    # Create dtype for structured array
    dtype_list = [(col, input_df[col].dtype) for col in input_df.columns]
    
    # Create structured array
    input_array = np.zeros(len(input_df), dtype=dtype_list)
    for col in input_df.columns:
        input_array[col] = input_df[col].values
    
    print(f"\n✅ Created numpy structured array with {len(input_array)} timesteps")
    print(f"   Input variables: {list(input_array.dtype.names)}")
    print(f"   Data type: {type(input_array)}")
    print(f"   Dtype:\n{input_array.dtype}")
    
    return input_array


# ============================================
# 3. RUN SIMULATION
# ============================================
print("🚀 START AHU FMU SIMULATION WITH REAL WEATHER DATA")
print("="*60)


# File paths
fmu_path = r"C:/Users/HON/AppData/Local/Temp/OpenModelica/OMEdit/AHU_FMU_Core.fmu"
weather_csv = "weather_data.csv"


# Load weather data
print("\n📊 Loading weather data...")
weather_data = load_weather_from_csv(weather_csv)


# Check data
print(f"\n📈 WEATHER DATA STATISTICS:")
print(f"   Time range: {weather_data['time'].min():.0f} - {weather_data['time'].max():.0f} seconds")
print(f"   Duration: {(weather_data['time'].max() - weather_data['time'].min())/86400:.1f} days")
print(f"   Temperature: {weather_data['TDryBul'].min()-273.15:.1f}°C to {weather_data['TDryBul'].max()-273.15:.1f}°C")
print(f"   Relative humidity: {weather_data['relHum'].min()*100:.1f}% to {weather_data['relHum'].max()*100:.1f}%")
print(f"   Max radiation: {weather_data['HGloHor'].max():.0f} W/m²")


# Prepare input for FMU
print("\n🔧 Preparing input time series...")
input_data = prepare_fmu_input(weather_data)


# Define simulation time
start_time = weather_data['time'].min()
stop_time = weather_data['time'].max()
step_size = 900
output_interval = 900


try:
    print("\n⏳ Running simulation...")
    print(f"   Time: {start_time} - {stop_time} seconds ({(stop_time-start_time)/86400:.1f} days)")
    print(f"   Time step: {step_size} seconds")
    print(f"   Output interval: {output_interval} seconds")
    
    result = simulate_fmu(
        fmu_path,
        start_time=start_time,
        stop_time=stop_time,
        step_size=step_size,
        logger=lambda s: print('logs:', s),
        output_interval=output_interval,
        fmi_type='ModelExchange',
        solver='CVode',
        relative_tolerance=1e-4,
        input=input_data,  # ✅ Pass numpy structured array
        output=[
            'T_SA', 'RH_SA', 'Vdot_SA',
            'T_zone', 'RH_zone', 'CO2_zone_ppm',
            'P_fan', 'Q_chiller', 'P_chiller',
            'P_fanEA', 'P_pump', 'P_total',
            'Q_heater', 'T_SA_afterCooling'
        ]
    )
    
    print(f"✅ SUCCESS! Simulated {len(result)} timesteps")
    
    # ============================================
    # 4. PLOT RESULTS
    # ============================================
    time_days = result['time'] / (24 * 3600)
    weather_time = weather_data['time'] / (24 * 3600)
    
    fig, axes = plt.subplots(4, 3, figsize=(18, 16))
    fig.suptitle('AHU SIMULATION WITH REAL WEATHER DATA - HANOI (UPGRADED MODEL)', 
                 fontsize=16, fontweight='bold', y=0.995)
    
    # === ROW 1: OUTDOOR CONDITIONS ===
    axes[0,0].plot(weather_time, weather_data['TDryBul']-273.15, 'orange', linewidth=1)
    axes[0,0].set_title('Outdoor Temperature [°C]', fontweight='bold')
    axes[0,0].set_ylabel('°C')
    axes[0,0].grid(True, alpha=0.3)
    
    axes[0,1].plot(weather_time, weather_data['relHum']*100, 'cyan', linewidth=1)
    axes[0,1].set_title('Outdoor Relative Humidity [%]', fontweight='bold')
    axes[0,1].set_ylabel('%')
    axes[0,1].grid(True, alpha=0.3)
    
    axes[0,2].plot(weather_time, weather_data['HGloHor'], 'gold', linewidth=1)
    axes[0,2].set_title('Solar Radiation [W/m²]', fontweight='bold')
    axes[0,2].set_ylabel('W/m²')
    axes[0,2].grid(True, alpha=0.3)
    
    # === ROW 2: TEMPERATURE ===
    T_zone_setpoint = 297.15  # 24°C
    
    axes[1,0].plot(time_days, result['T_zone']-273.15, 'b-', linewidth=1.5, label='Actual')
    axes[1,0].axhline(y=T_zone_setpoint-273.15, color='r', linestyle='--', alpha=0.5, label='Setpoint')
    axes[1,0].fill_between(time_days, 22, 26, alpha=0.1, color='green', label='Comfort zone')
    axes[1,0].set_title('Zone Temperature [°C]', fontweight='bold')
    axes[1,0].set_ylabel('°C')
    axes[1,0].grid(True, alpha=0.3)
    axes[1,0].legend(loc='best', fontsize=8)
    
    axes[1,1].plot(time_days, result['T_SA']-273.15, 'r-', linewidth=1.5, label='After heater')
    axes[1,1].plot(time_days, result['T_SA_afterCooling']-273.15, 'b--', linewidth=1, alpha=0.7, label='After cooling')
    axes[1,1].set_title('Supply Air Temperature (SA) [°C]', fontweight='bold')
    axes[1,1].set_ylabel('°C')
    axes[1,1].grid(True, alpha=0.3)
    axes[1,1].legend(loc='best', fontsize=8)
    
    axes[1,2].plot(time_days, result['RH_zone']*100, 'g-', linewidth=1.5)
    axes[1,2].fill_between(time_days, 40, 60, alpha=0.1, color='green', label='Comfort zone')
    axes[1,2].set_title('Zone Relative Humidity [%]', fontweight='bold')
    axes[1,2].set_ylabel('%')
    axes[1,2].grid(True, alpha=0.3)
    axes[1,2].legend(loc='best', fontsize=8)
    
    # === ROW 3: ENERGY ===
    axes[2,0].plot(time_days, result['P_total']/1000, 'darkred', linewidth=1.5, label='Total')
    axes[2,0].plot(time_days, result['P_fan']/1000, 'm--', linewidth=1, alpha=0.7, label='Supply fan')
    axes[2,0].plot(time_days, result['P_fanEA']/1000, 'purple', linestyle=':', linewidth=1, alpha=0.7, label='Exhaust fan')
    axes[2,0].set_title('Electrical Power [kW]', fontweight='bold')
    axes[2,0].set_ylabel('kW')
    axes[2,0].grid(True, alpha=0.3)
    axes[2,0].legend(loc='best', fontsize=7)
    
    axes[2,1].plot(time_days, result['Q_chiller']/1000, 'c-', linewidth=1.5)
    axes[2,1].set_title('Cooling Power [kW]', fontweight='bold')
    axes[2,1].set_ylabel('kW')
    axes[2,1].grid(True, alpha=0.3)
    axes[2,1].fill_between(time_days, 0, result['Q_chiller']/1000, alpha=0.2, color='cyan')
    
    axes[2,2].plot(time_days, result['P_chiller']/1000, 'orange', linewidth=1.5, label='Chiller')
    axes[2,2].plot(time_days, result['P_pump']/1000, 'brown', linestyle='--', linewidth=1, alpha=0.7, label='Pump')
    axes[2,2].set_title('Chiller & Pump Power [kW]', fontweight='bold')
    axes[2,2].set_ylabel('kW')
    axes[2,2].grid(True, alpha=0.3)
    axes[2,2].legend(loc='best', fontsize=8)
    
    # === ROW 4: AIR QUALITY & HEATER ===
    axes[3,0].plot(time_days, result['RH_SA']*100, 'purple', linewidth=1.5)
    axes[3,0].set_title('Supply Air Humidity [%]', fontweight='bold')
    axes[3,0].set_ylabel('%')
    axes[3,0].set_xlabel('Time [days]')
    axes[3,0].grid(True, alpha=0.3)
    
    axes[3,1].plot(time_days, result['Vdot_SA']*3600, 'brown', linewidth=1.5)
    axes[3,1].set_title('Supply Air Flow Rate [m³/h]', fontweight='bold')
    axes[3,1].set_ylabel('m³/h')
    axes[3,1].set_xlabel('Time [days]')
    axes[3,1].grid(True, alpha=0.3)
    
    axes[3,2].plot(time_days, result['CO2_zone_ppm'], 'darkgreen', linewidth=1.5)
    axes[3,2].axhline(y=1000, color='r', linestyle='--', alpha=0.5, label='Max 1000 ppm')
    axes[3,2].axhline(y=400, color='gray', linestyle=':', alpha=0.5, label='Outdoor 400 ppm')
    axes[3,2].set_title('Zone CO2 Concentration [ppm]', fontweight='bold')
    axes[3,2].set_ylabel('ppm')
    axes[3,2].set_xlabel('Time [days]')
    axes[3,2].grid(True, alpha=0.3)
    axes[3,2].legend(loc='best', fontsize=8)
    
    plt.tight_layout()
    plt.savefig('ahu_weather_results.png', dpi=150, bbox_inches='tight')
    print("\n💾 Saved plot: ahu_weather_results.png")
    plt.show()
    
    # ============================================
    # 5. RESULTS STATISTICS
    # ============================================
    print("\n" + "="*60)
    print("📊 SIMULATION RESULTS STATISTICS")
    print("="*60)
    
    print("\n🌡️  TEMPERATURE:")
    print(f"   Zone:            {(result['T_zone']-273.15).min():.1f} - {(result['T_zone']-273.15).max():.1f}°C (Avg: {(result['T_zone']-273.15).mean():.1f}°C)")
    print(f"   SA (final):      {(result['T_SA']-273.15).min():.1f} - {(result['T_SA']-273.15).max():.1f}°C (Avg: {(result['T_SA']-273.15).mean():.1f}°C)")
    print(f"   SA (after cool): {(result['T_SA_afterCooling']-273.15).min():.1f} - {(result['T_SA_afterCooling']-273.15).max():.1f}°C (Avg: {(result['T_SA_afterCooling']-273.15).mean():.1f}°C)")
    print(f"   Outdoor:         {(weather_data['TDryBul']-273.15).min():.1f} - {(weather_data['TDryBul']-273.15).max():.1f}°C")
    
    print("\n💧 RELATIVE HUMIDITY:")
    print(f"   Zone: {(result['RH_zone']*100).min():.1f} - {(result['RH_zone']*100).max():.1f}% (Avg: {(result['RH_zone']*100).mean():.1f}%)")
    print(f"   SA:   {(result['RH_SA']*100).min():.1f} - {(result['RH_SA']*100).max():.1f}% (Avg: {(result['RH_SA']*100).mean():.1f}%)")
    
    print("\n🌫️  AIR QUALITY:")
    print(f"   CO2: {result['CO2_zone_ppm'].min():.0f} - {result['CO2_zone_ppm'].max():.0f} ppm (Avg: {result['CO2_zone_ppm'].mean():.0f} ppm)")
    
    print("\n⚡ ENERGY:")
    print(f"   Supply fan avg:      {result['P_fan'].mean():.1f} W")
    print(f"   Exhaust fan avg:     {result['P_fanEA'].mean():.1f} W")
    print(f"   Pump avg:            {result['P_pump'].mean():.1f} W")
    print(f"   Cooling power avg:   {result['Q_chiller'].mean()/1000:.1f} kW")
    print(f"   Chiller power avg:   {result['P_chiller'].mean()/1000:.1f} kW")
    print(f"   Heater power avg:    {result['Q_heater'].mean():.1f} W")
    print(f"   Total power avg:     {result['P_total'].mean()/1000:.2f} kW")
    
    total_days = (stop_time - start_time) / 86400
    total_energy = result['P_total'].sum() * output_interval / 3600000
    print(f"   Total energy consumed: {total_energy:.1f} kWh ({total_energy/total_days:.1f} kWh/day)")
    
    # Save results to CSV
    results_df = pd.DataFrame({
        'time_days': time_days,
        'T_zone_C': result['T_zone'] - 273.15,
        'T_SA_C': result['T_SA'] - 273.15,
        'T_SA_afterCooling_C': result['T_SA_afterCooling'] - 273.15,
        'RH_zone_%': result['RH_zone'] * 100,
        'RH_SA_%': result['RH_SA'] * 100,
        'CO2_ppm': result['CO2_zone_ppm'],
        'P_fan_W': result['P_fan'],
        'P_fanEA_W': result['P_fanEA'],
        'P_pump_W': result['P_pump'],
        'Q_chiller_kW': result['Q_chiller'] / 1000,
        'P_chiller_kW': result['P_chiller'] / 1000,
        'Q_heater_W': result['Q_heater'],
        'P_total_kW': result['P_total'] / 1000,
        'Vdot_SA_m3h': result['Vdot_SA'] * 3600
    })
    
    results_df.to_csv('ahu_results.csv', index=False)
    print("\n💾 Saved results: ahu_results.csv")
    print("\n" + "="*60)
    
except Exception as e:
    print(f"\n❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    
    print("\n💡 TROUBLESHOOTING TIPS:")
    print("   1. Verify FMU file exists")
    print("   2. Check input variable names in FMU:")
    print("      Run: from fmpy import dump; dump('HVAC_FMU_FIXED_INIT_5_AHU_FMU_Core.fmu')")
    print("   3. Ensure input column names match FMU input variable names")
    print("   4. Try simulating just 1 day first: stop_time = start_time + 86400")


🚀 START AHU FMU SIMULATION WITH REAL WEATHER DATA

📊 Loading weather data...
✅ Loaded 8760 weather timesteps from weather_data.csv

Columns in file: ['time', 'TDryBul', 'relHum', 'pAtm', 'HGloHor', 'HDifHor', 'winSpe', 'winDir']

Sample data:
      time  TDryBul  relHum    pAtm  HGloHor  HDifHor  winSpe  winDir
0      0.0   288.05    0.82  101900        0        0     0.0       0
1   3600.0   286.05    0.82  102000        0        0     0.0       0
2   7200.0   284.55    0.82  102100        0        0     0.0       0
3  10800.0   283.65    0.83  102200        0        0     0.0       0
4  14400.0   283.05    0.84  102200        0        0     1.0       0

📈 WEATHER DATA STATISTICS:
   Time range: 0 - 31618800 seconds
   Duration: 366.0 days
   Temperature: 9.0°C to 37.5°C
   Relative humidity: 23.0% to 100.0%
   Max radiation: 1021 W/m²

🔧 Preparing input time series...

✅ Created numpy structured array with 8760 timesteps
   Input variables: ['time', 'TDryBul_in', 'relHum_in', 'pAtm_i

Traceback (most recent call last):
  File "C:\Users\HON\AppData\Local\Temp\ipykernel_11156\1186599765.py", line 131, in <module>
    result = simulate_fmu(
  File "c:\Users\HON\anaconda3\envs\fmi\lib\site-packages\fmpy\simulation.py", line 759, in simulate_fmu
    result = simulateME(model_description, fmu, start_time, stop_time, solver, step_size, relative_tolerance, start_values, apply_default_start_values, input, output, output_interval, record_events, timeout, step_finished, validate, set_stop_time)
  File "c:\Users\HON\anaconda3\envs\fmi\lib\site-packages\fmpy\simulation.py", line 1059, in simulateME
    state_event, roots_found, time = solver.step(time, t_next)
  File "c:\Users\HON\anaconda3\envs\fmi\lib\site-packages\fmpy\sundials\__init__.py", line 178, in step
    raise RuntimeError("CVode error (code %s) in module %s, function %s: %s" % self.error_info)
RuntimeError: CVode error (code -1) in module CVODE, function CVode: At t = 31.6394, mxstep steps taken before reaching tout